# Exploration of the Construction of Liquid Fuel Bipropellant Rocket Engines Utilizing Additive Manufacturing Processes


## Abstract:

The Portland State Aerospace Society (PSAS) is an engineering student group and citizen science project at Portland State University dedicated to building low-cost, open-source, open-hardware rockets and avionics systems. The group’s stated long term goal is to place a 1 kg cubesat into low Earth orbit with their own launch vehicle. One step needed to achieve this goal is to transition the current rocket design from a solid motor to a liquid fuel engine. The liquid propelled rocket engine project is being conducted as part of a mechanical engineering senior capstone project at Portland State University. This project is on track to being completed by June of 2016.

The complexity and cost of building a liquid fuel rocket engine typically makes such devices unobtainable for a majority of parties interested in their construction. Until recently, manufacturing processes and techniques limited the geometries available to the designer and rendered such engines cost prohibitive as options for inexpensive orbital space flight. Advances in additive manufacturing technologies provide the potential to prototype complex geometries on a lower budget and with shorter lead times which would be considered unobtainable with traditional manufacturing methods. Furthermore, bipropellant liquid fuels offer many complex engineering considerations; the full analysis of which may not be within the design ability of many amateur builds. It is therefore advantageous to develop techniques for additive manufacturing rapid prototyping to make the study of bipropellant fuels more accessible.

Explored herein is the process of designing and testing a liquid bipropellant engine on the scale of 50-1000 lbf of thrust using liquid oxygen (LOX) and ethanol as propellants. A low cost pintle injector and accompanying regeneratively cooled thrust chamber is developed using a combination of traditional manufacturing techniques and additive processes. Equations have been determined to describe the more complex geometries of the nozzle contour and the sizing of other important components. Figure 1 shows the preliminary design of the combustion chamber and nozzle with cooling channels. Heat transfer analysis is being conducted to determine the type of metal to be selected for the nozzle and cooling chamber, most likely a high-temperature steel such as inconel.


Figure 1: Combustion chamber and nozzle preliminary design

In order to achieve a simple and easy to manufacture pintle injector design, the regenerative cooling channel interface, fuel manifold and most of the injector plate will be part of the additively manufactured combustion chamber. The design will accommodate easy installation of different injector types to allow for testing of different injector and spray configurations. Cold flow tests will be conducted to study the viscous losses in 3D-printed metal cooling channels, manifold system, and pintle injector designs.

An engine test-stand has been constructed and a pressure fed fuel system with actuated valves is being developed for fuel delivery. Pressure transducers and thermocouples will be ported into the final engine design to collect pressure and temperature data in the nozzle and combustion chamber. Analytical solutions and empirical data will be compared against simulations carried out by computational fluid dynamics (CFD) simulations. CFD simulations will be conducted using Loci/CHEM and Star-CCM+ simulation software.

#### Initial Design Considerations

Parameters must be selected. The generation of force is the desire of a rocket engine. the force equation is:

$$ F = \frac{\dot{w}}{g} V_{e} + A_{e}(P_{e}-P_{a}) $$

This is important when designing engines for flight, but when testing engines $P_{e}$ is optimally expanded and assumed to be atmospheric pressure (14.7 psia) and the equation becomes:

$$ F = \frac{\dot{w}}{g} V_{e} $$

For engines in flight the effective exhaust velocity, $c$, can be used, and equation X can be expressed as:

$$F = c\frac{\dot{W}}{g} $$

Where c is defined as

$$c=V_{e} + A_{e}(P_{e}-P_{a})\frac{g}{\dot{W}} $$

which changes with altitude.

For a desired force, there must then be selected an exit velocity to find the necessary flow rate, $\dot{w}$. The theoretical exit velocity is defined as:

$$V_{e} = \sqrt{\frac{2g\gamma}{\gamma-1}RT_{i}\bigg[1-\bigg(\frac{P_{e}}{P_{i}}\bigg)\bigg]^\frac{\gamma-1}{\gamma}+v_{i}^2} $$

Because the inlet velocity is very small, it is assumed to be zero, this gives the following:

$$V_{e} = \sqrt{\frac{2g\gamma}{\gamma-1}R(T_{c})_{ns}\bigg[1-\bigg(\frac{P_{e}}{(P_{c})_{ns}}\bigg)\bigg]^\frac{\gamma-1}{\gamma}} $$

which is dependent on the propellants, chamber pressure which should be chosen for the design, $P_{i}$, and the mach number at the nozzle inlet, Mi, which must be calculated iteratively.

Flame temperature and gamma can be obtained for a given propellant combination by using NASA's [CEArun](https://cearun.grc.nasa.gov/) tool. [link to portion in document which describes how to use cearun](internallinkhere)



In [35]:
import math
from mpmath import *
from IPython.display import display
from ipywidgets import widgets

#Define Constants
g     = 32.2  # gravitational constant in ft/s^2
J     = 778   # Energy conversion factor (ft-lb/Btu)
Regas = 1544  # Gas constant (ft/degR)
Rgas  = 8.314 # Gas constant (J/mol/K)

In [36]:
#initial design considerations inputs
F_        = widgets.Text("250",           description="Force, F (lbf)",                              width=60 )
M_        = widgets.Text("23.8",          description="Molecular Weight, (lb/lb-mol)",               width=100)
Pa_       = widgets.Text("14.7",          description="Atmospheric Pressure, (lbf/in^2)",            width=60 )
Pe_       = widgets.Text("14.7",          description="Exit Pressure, Pe (lbf/in^2) ",               width=60 )
Pinj_     = widgets.Text("350",           description="Pressure at injector, (lb/in^2)",             width=60 )
rw_       = widgets.Text("1.2",           description="(oxidizer/fuel) Weight mixture ratio",        width=60 )
Tamb_     = widgets.Text("519.67",        description="Ambient temperature, (deg R)",                width=60 )
Tcns_     = widgets.Text("5487.1",        description="Flow temperature at nozzle inlet, (deg R)",   width=80 )
epsilonc_ = widgets.Text("10",             description="Contraction ratio",                           width=60 )
gamma_    = widgets.Text("1.12",          description="Specific heat ratio",                         width=60 )

display(F_)
display(M_)
display(Pa_)
display(Pe_)
display(Pinj_)
display(rw_)
display(Tamb_)
display(Tcns_)
display(epsilonc_)
display(gamma_)

In [37]:
F        = float(F_.value)        #Nozzle force
M        = float(M_.value)        #Molecular Weight
Pa       = float(Pa_.value)       #Atmospheric Pressure
Pe       = float(Pe_.value)       #Pressure at exit
Pinj     = float(Pinj_.value)     #Pressure at injector
rw       = float(rw_.value)       #Weight mixture ratio
Tamb     = float(Tamb_.value)     #Ambient temperature
Tcns     = float(Tcns_.value)     #Nozzle stagnation temperature
epsilonc = float(epsilonc_.value) #Contraction ratio
gamma    = float(gamma_.value)    #Specific Heat Ratio

In [38]:
#program itterative process to find mach number at the inlet.
Mi_current = .3 # a reasonable guess for mach number at inlet
Mi_last = 0

while (abs(Mi_current-Mi_last)>0.0001):
    Mi_last = Mi_current
    Mi_current = math.sqrt(((1+(gamma-1)/2*Mi_last)/((gamma+1)/2))**((gamma+1)/(gamma-1)))/epsilonc

Mi = Mi_current

In [39]:
#Assumptions

Pcinj    = Pinj                   #Chamber total pressure is equal to injector pressure
Minj     = 0                      #Mach number at injector assumed to be zero
R        = Regas/M                #Gas constant for the flow
Tci      = Tamb                   #Fuel holding temperature is equal to ambient temp
Vinj     = 0                      #Injector velocity is zero


In [40]:
#Calculations

Pcns     = Pcinj*(1+((gamma-1)/2)*Mi**2)**(gamma/(gamma-1))/(1+gamma*Mi**2)           #Nozzle stagnation pressure
Pi       = Pinj/(1+gamma*Mi**2)                                                       #Pressure at inlet
Pt       = Pcns*(2/(gamma+1))**(gamma/(gamma-1))                                      #Pressure at throat
Vt       = math.sqrt(((2*g*gamma)/(gamma+1))*R*Tcns)                                  #Velocity at throat
Ve       = math.sqrt(((2*g*gamma)/(gamma-1))*R*Tcns*(1-(Pe/Pi)**((gamma-1)/gamma)))   #Velocity at exit
wdot     = F*g/Ve                                                                     #Propellant flow rate
wdotf    = 1/(1+rw)*wdot                                                              #Fuel mass flow rate
wdoto    = wdot-wdotf                                                                 #Ox mass flow rate
At       = wdot/(Pcns*math.sqrt(g*gamma*(2/(gamma+1))**((gamma+1)/(gamma-1))/R/Tcns)) #Area of throat
epsilon  = ((2/(gamma+1))**(1/(gamma-1))*(Pcns/Pe)**(1/gamma)
               /math.sqrt((gamma+1)/(gamma-1)*(1-(Pe/Pcns))**((gamma-1/gamma))))      #Expansion ratio
Ae       = At*epsilon                                                                 #Area of exit
c        = Ve+Ae*(Pe-Pa)*(g/wdot)                                                     #Effective exhaust velocity
Ti       = Tcns/(1+.5*(gamma-1)*Mi**2)

In [41]:
#Print results
print("Pressures:")
print("Inlet,          Pi (psi):       %.2f"   % Pi)
print("Throat,         Pt (psi):       %.2f\n" % Pt)
print("Velocities:")
print("Throat,         Vt (ft/sec):    %.2f"   % Vt)
print("Exit,           Vt (ft/sec):    %.2f\n" % Ve)
print("Mass Flow Rates:")
print("Total,          wdot (lb/sec):  %.2f"   % wdot)
print("Fuel,           wdotf (lb/sec): %.2f"   % wdotf)
print("Oxidizer,       wdoto (lb/sec): %.2f\n" % wdoto)

Pressures:
Inlet,          Pi (psi):       348.51
Throat,         Pt (psi):       202.75

Velocities:
Throat,         Vt (ft/sec):    3480.09
Exit,           Vt (ft/sec):    7845.22

Mass Flow Rates:
Total,          wdot (lb/sec):  1.03
Fuel,           wdotf (lb/sec): 0.47
Oxidizer,       wdoto (lb/sec): 0.56



In [42]:
#Define User Inputs for Equation Variables, assign values to default values
a_        = widgets.Text("0.000008065",   description="Thermal expansion ratio nozzle material",     width=200)
ARhb_     = widgets.Text("6",             description="Cooling channel aspect ratio at throat",      width=60 )
Bpf_      = widgets.Text("632.07",        description="Boiling Point of Fuel, Bpf (deg K)",          width=60 )
Cpf_      = widgets.Text(".71157",        description="Fuel specific heat, Cpf (Btu/lb-deg F)",      width=100)
DelPi_    = widgets.Text("87.5",          description="Injector pressure drop, DelPi (lbf/in^2)",    width=60 )
E_        = widgets.Text("25450000",      description="Elastic modulus of nozzle material, E (psi)", width=60 )

Hvapf_    = widgets.Text("38600",         description="Fuel heat of vaporization, Hvapf (KJ/mol)",   width=60 )
k_        = widgets.Text(".000225694",    description="Wall thermal conductivity, k (Btu/in^2-s-F)", width=60 )
Lc_       = widgets.Text("3.5",           description="Chamber Length, Lc (in) ",                    width=60 )
Lstar_    = widgets.Text("50",            description="Characteristic Chamber Length, L* (in) ",     width=60 )

n_        = widgets.Text("100",           description="Number of cooling channels",                  width=60 )
nc_       = widgets.Text("1.1",           description="Nucleate boiling factor of safety, nc",       width=60 )

Pco_      = widgets.Text("550",           description="Initial fuel pressure, Pco (lbf/in^2)",       width=60 )





Tt_       = widgets.Text("5232.4",        description="Throat Temperature (deg R)",                  width=80 )
Twg_      = widgets.Text("2459.67",          description="Maximum Wall Temperature",                    width=60 )
v_        = widgets.Text("0.274",         description="Poisson's ratio nozzle material",             width=60 )
eta_      = widgets.Text("0.90",          description="Combustion efficiency",                       width=60 )




#Display user input text boxes
display(a_)
display(ARhb_)
display(Bpf_)
display(Cpf_)
display(DelPi_)
display(E_)

display(Hvapf_)
display(k_)
display(Lc_)
display(Lstar_)

display(n_)
display(nc_)

display(Pco_)





display(Tt_)
display(Twg_)
display(v_)
display(eta_)



In [43]:
#Convert string entries to floats

a        = float(a_.value)        #Thermal expansion ratio of nozzle material
ARhb     = float(ARhb_.value)     #Cooling channel aspect ratio
Bpf      = float(Bpf_.value)      #Boiling Point of fuel
Cpf      = float(Cpf_.value)      #Fuel specific heat at constant pressure
DelPi    = float(DelPi_.value)    #Pressure drop across injector
E        = float(E_.value)        #Elastic modulus of nozzle material

Hvapf    = float(Hvapf_.value)    #Fuel heat of vaporization
k        = float(k_.value)        #Wall thermal conductivity
Lc       = float(Lc_.value)       #Length of Combustion Chamber
Lstar    = float(Lstar_.value)    #Characteristic Chamber Length

n        = float(n_.value)        #Number of cooling channels
nc       = float(nc_.value)       #Nucleate boiling factor of safety

Pco      = float(Pco_.value)      #Initial fuel pressure





Tt       = float(Tt_.value)       #Temperature of throat flow
Twg      = float(Twg_.value)      #Maximum Wall Temperature
v        = float(v_.value)        #Poisson's ratio nozzle material
eta      = float(eta_.value)      #Combustion efficiency










In [44]:
#Print current parameters

In [45]:
#Performance Parameters

Is     = F/wdot                                                               #Specific Impulse
Istc   = c/g                                                                  #Thrust chamber specific impulse
wdottc = F/Istc*eta                                                           #Thrust chamber propellant flow rate
cstar  = (math.sqrt(g*gamma*R*Tcns)/gamma
             /math.sqrt((2/(gamma+1))**((gamma+1)/(gamma-1))))                #Characteristic Velocity
Cf     = (math.sqrt(2*gamma**2/(gamma-1)*(2/(gamma+1))**((gamma+1)/(gamma-1))
             *(1-(Pe/Pcns)**((gamma-1)/gamma)))+epsilon*((Pe-Pa)/Pcns))       #Thrust Coefficient

In [46]:
#Print Performance Parameters

In [47]:
#Thrust Chamber Layout

Vc = Lstar*At #Chamber volume
Ac = epsilonc*At #Chamber cross sectional area


In [48]:
#print thrust chamber layout

In [49]:
#Heat Transfer
Pr     = 4*gamma/(9*gamma-5)                                     #Prandtl number
mucc   = (46.6*10**-10)*M**0.5*Tcns                              #Viscosity in the combustion chamber
mut    = (46.6*10**-10)*M**0.5*Tt                                #Viscosity in the throat
rlam   = Pr**0.5                                                 #Laminar flow local recovery factor
rturb  = Pr**0.33                                                #Turbulent flow local recovery factor
Reffcc = ((1+rturb*((gamma-1)/2)*Mi**2)/(1+((gamma-1)/2)*Mi**2))
Refft  = ((1+rturb*((gamma-1)/2))/(1+((gamma-1)/2)))
Tawi   = Tcns*Reffcc                                             #Adiabatic wall temperature at inlet
Tawt   = Tcns*Refft                                              #Adiabatic wall temperature at throat
rt     = math.sqrt(At/math.pi)                                   #Radius of throat
re     = math.sqrt(Ae/math.pi)                                   #Radius of exit
rmean  = rt*(1.5+.382)/2                                         #Mean throat curvature
sigmat = (1/((.5*Twg/Tcns*(1+(gamma-1)/2)+.5)**0.68              #Correction factor for property variations across BL
             *(1+(gamma+1)/2)**0.12))                            #specified at throat
sigmai = (1/((.5*Twg/Tcns*(1+(gamma-1)/2*Mi**2)+.5)**0.68              #Correction factor for property variations across BL
             *(1+(gamma+1)/2*Mi**2)**0.12))                            #specified at inlet
Cp     = gamma*R/(gamma-1)/J                                     #Specific heat at constant pressure
hg     = ((0.026/(2*rt)**0.2*(mucc**0.2*Cp/Pr**0.6)
           *(Pcns*g/cstar)**0.8*(2*rt/rmean)**0.1)*sigmat)       #heat transfer coefficient at throat
q      = hg*(Tawt-Twg)                                           #required heat flux
Tcc    = 1.8/(Rgas/Hvapf*math.log((Pi+DelPi)/Pa)+1/Bpf)          #Critical temperature of fuel coolant
Twc    = Tcc/nc                                                  #Maximum coolant wall temperature
Qc     = wdotf*Cpf*(Twc-Tci)                                     #Coolant capacity
Tbulk  = (Twc + Tci)/2                                           #Coolant bulk temp
t      = k/q*(Twg-Tcc)                                           #Wall thickness
hc     = q/(Twc-Tbulk)                                           #Coolant side heat transfer coefficient
H      = 1/(1/hg+t/k+1/hc)                                       #Overall heat transfer coefficient


In [50]:
#print heat transfer
print(Tcc)
print(Qc)
print(Twc)
print(Tbulk)
print(Tci)

778.468567616456
62.40361466897919
707.6986978331418
613.6843489165708
519.67


In [51]:
#cooling channel geometry
bt     = (2*rt+t)*math.pi/n-t     #base width at throat
be     = (2*re+t)*math.pi/n-t     #base width at exit
rccht  = bt/2                     #cooling channel effective radius at throat
rcche  = be/2                     #cooling channel effective radius at nozzle exit
d      = 2*ARhb*bt**2/(ARhb+1)/bt #cooling channel hydraulic diameter

#can we calculate the dynamic viscosity of ethanol for a given bulk temp?

muf = 2.7998705 *10**-5 
rhof = 0.031972653 #density of 70% ethanol

Vco    = 4000/rhof/d*muf          #needed velocity for turbulent flow in in/s
#Vco    = 
#mubulk = #coolant viscosity at bulk temp
#muw    = #coolant viscosity at sidewall temp
#kf     = #fuel thermal conductivity

print(Vco)

200.89014566922683


In [52]:
#wall stresses
Ste    =(Pco-Pe)*rcche/t+E*a*q*t/2/(1-v)/k #combined tangential stress at nozzle exit
Sce    =(Pco-Pe)*re/t+E*a*q*t/2/(1-v)/k    #maximum compressive stress as coaxial shell design
print(t)

0.015075389359098624


In [ ]:
rcc = math.sqrt((At*epsilonc)/math.pi)
#Parametric Equations

#calculate parameters

tout   = 2*t # calculate a better outer thickness or place in input parameters
theta  = 30
thetaN = 30
A1     = math.tan(math.radians(90-thetaN))/(2*(rt-rt*.382*(1-math.cos(math.radians(thetaN)))))
A2     = -(.382*rt*math.sin(math.radians(thetaN)))+A1*(rt+.382*rt*(1-math.cos(math.radians(thetaN))))**2
X1     = rt+.382*rt*(1-math.cos(math.radians(thetaN)))

#calculate total cooling channel cross sectional area
A_c    = .179341542
#calculate this value better...

In [62]:
print("Parametric Equations:\n")
print("Internal Contour:\n")

print("Bell")
print("x(t):")
print(" t")
print("y(t):")
print(" %f * t^2 - %f" % (A1,A2) )
print("Parameters:" )
print("t1:")
print(" %f" % X1 )
print("t2:")
print(" %f\n" % (re-0.0001) )

A3 = 1.382*rt
A4 = 0.382*rt

print("Throat, rapid expansion")
print("x(t):")
print(" %f * (1- (.382/1.382) * cos(t*3.14159/180))" % A3 )
print("y(t):")
print(" %f * sin(t*3.14159/180)" % A4 )
print("Parameters:" )
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (thetaN-0.001) )

A5 = 2.5*rt
A6 = -1.5*rt

print("Throat, rapid contraction")
print("x(t):")
print(" %f * (1 - 0.6 * cos(t*3.14159/180))" % A5 )
print("y(t):")
print(" %f * sin(t*3.14159/180)" % A6 )
print("Parameters:" )
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (theta-0.001) )

A7 = -math.tan(math.pi/2-math.radians(theta))
A8 = (math.tan(math.pi/2-math.radians(theta))*2.5*rt*(1-0.6*math.cos(math.radians(theta))))-1.5*rt*math.sin(math.radians(theta))
p1 = 2.5*rt*(1-0.6*math.cos(math.radians(theta)))
p2 = rcc-1.5*rt*(1-math.cos(math.radians(theta)))

print("Linear contraction")
print("x(t):")
print(" t")
print("y(t):")
print(" %f * t + %f" % (A7,A8))
print("Parameters:" )
print("t1:")
print(" %f" % (p1+0.0001) )
print("t2:")
print(" %f\n" % (p2-0.0001) )

A9  = 1.5*rt
A10 = 1.5*rt
A11 = (math.tan(2*math.pi-(math.pi/2-math.radians(theta))))*(rcc-1.5*rt*(1-math.cos(math.radians(theta))))+(-math.tan(2*math.pi-(math.pi/2-math.radians(theta)))*(2.5*rt*(1-0.6*math.cos(math.radians(theta))))-1.5*rt*math.sin(math.radians(theta)))-1.5*rt*math.sin(math.radians(theta))

print("Inlet")
print("x(t):")
print(" %f - %f * (1 - cos(t*3.14159/180))" % (rcc,A9))
print("y(t):")
print(" %f * sin(t*3.14159/180) + %f" % (A10,A11))
print("Parameters:")
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (theta-0.0001) )

Parametric Equations:

Internal Contour:

Bell
x(t):
 t
y(t):
 2.314970 * t^2 - 0.322342
Parameters:
t1:
 0.414455
t2:
 0.621971

Throat, rapid expansion
x(t):
 0.544890 * (1- (.382/1.382) * cos(t*3.14159/180))
y(t):
 0.150614 * sin(t*3.14159/180)
Parameters:
t1:
 0.001
t2:
 29.999000

Throat, rapid contraction
x(t):
 0.985691 * (1 - 0.6 * cos(t*3.14159/180))
y(t):
 -0.591414 * sin(t*3.14159/180)
Parameters:
t1:
 0.001
t2:
 29.999000

Linear contraction
x(t):
 t
y(t):
 -1.732051 * t + 0.524437
Parameters:
t1:
 0.473611
t2:
 1.167476

Inlet
x(t):
 1.246811 - 0.591414 * (1 - cos(t*3.14159/180))
y(t):
 0.591414 * sin(t*3.14159/180) + -1.793571
Parameters:
t1:
 0.001
t2:
 29.999900



In [63]:
print("Cooling Channel, inner:\n" )

A12 = 1/(4*A1**2)

print("Bell")
print("x(t):")
print(" t + ( %f * t / ( t^2 + %f )^0.5)" % (t,A12))
print("y(t):")
print(" %f *t^2 - %f + ( %f / ( %f *( t^2 + %f )^0.5))" % (A1,A2,-t,2*A1,A12))
print("Parameters:")
print("t1:")
print(" %f" % (X1+0.0001) )
print("t2:")
print(" %f\n" % (re-0.0001) )

print("Rapid Expansion")
print("x(t):")
print(" %f * (1- (.382/1.382) * cos(t*3.14159/180)) + ( %f * cos(t*3.14159/180))" % (A3,t) )
print("y(t):")
print(" %f * sin(t*3.14159/180) + (%f *sin(t*3.14159/180))" % (A4,-t) )
print("Parameters:" )
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (thetaN-0.001) )



print("Rapid Contraction")
print("x(t):")
print(" %f * (1 - 0.6 * cos(t*3.14159/180)) + ( %f * cos(t*3.14159/180))" % (A5,t) )
print("y(t):")
print(" %f * sin(t*3.14159/180) + (%f *sin(t*3.14159/180))" % (A6,t) )
print("Parameters:" )
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (theta-0.001) )

A13 = math.cos(math.radians(theta))*t
A14 = math.sin(math.radians(theta))*t

print("Linear contraction")
print("x(t):")
print(" t + %f" % A13)
print("y(t):")
print(" %f * t + %f" % (A7,A8+A14))
print("Parameters:" )
print("t1:")
print(" %f" % (p1+0.0001) )
print("t2:")
print(" %f\n" % (p2-0.0001) )

print("Inlet")
print("x(t):")
print(" %f - %f * (1 - cos(t*3.14159/180)) + ( %f * cos(t*3.14159/180))" % (rcc,A9,t))
print("y(t):")
print(" %f * sin(t*3.14159/180) + %f + (%f *sin(t*3.14159/180))" % (A10,A11,t))
print("Parameters:")
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (theta-0.0001) )

Cooling Channel, inner:

Bell
x(t):
 t + ( 0.015075 * t / ( t^2 + 0.046650 )^0.5)
y(t):
 2.314970 *t^2 - 0.322342 + ( -0.015075 / ( 4.629941 *( t^2 + 0.046650 )^0.5))
Parameters:
t1:
 0.414555
t2:
 0.621971

Rapid Expansion
x(t):
 0.544890 * (1- (.382/1.382) * cos(t*3.14159/180)) + ( 0.015075 * cos(t*3.14159/180))
y(t):
 0.150614 * sin(t*3.14159/180) + (-0.015075 *sin(t*3.14159/180))
Parameters:
t1:
 0.001
t2:
 29.999000

Rapid Contraction
x(t):
 0.985691 * (1 - 0.6 * cos(t*3.14159/180)) + ( 0.015075 * cos(t*3.14159/180))
y(t):
 -0.591414 * sin(t*3.14159/180) + (0.015075 *sin(t*3.14159/180))
Parameters:
t1:
 0.001
t2:
 29.999000

Linear contraction
x(t):
 t + 0.013056
y(t):
 -1.732051 * t + 0.531975
Parameters:
t1:
 0.473611
t2:
 1.167476

Inlet
x(t):
 1.246811 - 0.591414 * (1 - cos(t*3.14159/180)) + ( 0.015075 * cos(t*3.14159/180))
y(t):
 0.591414 * sin(t*3.14159/180) + -1.793571 + (0.015075 *sin(t*3.14159/180))
Parameters:
t1:
 0.001
t2:
 29.999900



In [64]:
print("Cooling Channel, outer:\n" )

A15 = A_c/math.pi

print("Bell")
print("x(t):")
print(" t + ( %f * t / ( t^2 + %f )^0.5) + ( t / ( t^2 + %f )^0.5) * ( -t + (t^2 + %f * sin(2*3.14159 - arctan(%f*t)))^(.5)) / (sin(2*3.14159-arctan(%f*t)))" % (t,A12,A12,A15,-2*A1,-2*A1))
print("y(t):")
print(" %f *t^2 - %f + ( %f / ( %f *( t^2 + %f )^0.5)) + ( 1 / ( %f * (t^2+ %f )^0.5)) * ( -t + ( t^2 + %f * sin(2*3.14159-arctan(%f*t)))^0.5) / ( sin(2*3.14159 - arctan(%f*t)))" % (A1,A2,-t,2*A1,A12,-2*A1,A12,A15,-2*A1,-2*A1))
print("Parameters:")
print("t1:")
print(" %f" % (X1+0.0001) )
print("t2:")
print(" %f\n" % (re-0.0001) )

print("Rapid Expansion")
print("x(t):")
print(" %f * (1- (.382/1.382) * cos(t*3.14159/180)) + ( %f * cos(t*3.14159/180)) + ( -( %f * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) ) + (( %f * (1 - (0.382)/(1.382) * cos(t*3.14159/180) ))^2 + %f * cos(t*3.14159/180))^0.5)" % (A3,t,A3,A3,A15)     )
print("y(t):")
print(" %f * sin(t*3.14159/180) + (%f *sin(t*3.14159/180)) + ( -( %f * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) ) + ( (%f * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) )^2 + %f * cos(t*3.14159/180) )^0.5 ) * ( -tan(t*3.14159/180) )" % (A4,-t,A3,A3,A15)     )
print("Parameters:" )
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (thetaN-0.001) )

print("Rapid Contraction")
print("x(t):")
print(" %f * (1 - 0.6 * cos(t*3.14159/180)) + ( %f * cos(t*3.14159/180)) + ( -( %f * ( 1 - 0.6 * cos(t*3.14159/180) ) ) + ( ( %f * ( 1 - 0.6 * cos(t*3.14159/180) ) )^2 + ( %f ) * cos(t*3.14159/180) )^0.5 )" % (A5,t,A5,A5,A15)     )
print("y(t):")
print(" %f * sin(t*3.14159/180) + (%f *sin(t*3.14159/180)) + ( -( %f * ( 1 - 0.6 * cos(t*3.14159/180) ) ) + ( ( %f * ( 1 - 0.6 * cos(t*3.14159/180) ) )^2 + %f * cos(t*3.14159/180) )^0.5 ) * ( tan(t*3.14159/180) )"             % (A6,t,A5,A5,A15)     )
print("Parameters:" )
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (theta-0.001) )

A16 = math.cos(math.radians(theta))*A15
A17 = math.tan(math.pi/2-math.radians(theta))*2.5*rt*(1-0.6*math.cos(math.radians(theta)))
A18 = -1.5*rt*math.sin(math.radians(theta))
A19 = A15 * math.cos(math.radians(theta))
A20 = math.tan(math.radians(theta))

print("Linear contraction")
print("x(t):")
print(" t + %f + ( -t + ( t^2 + %f )^0.5 )" % (A13,A16))
print("y(t):")
print(" %f *t + %f + %f + ( -t + ( t^2 + %f )^0.5 ) * ( %f ) + ( %f )" % (A7,A17,A18,A19,A20,A14))
print("Parameters:" )
print("t1:")
print(" %f" % (p1+0.0001) )
print("t2:")
print(" %f\n" % (p2-0.0001) )

print("Inlet")
print("x(t):")
print(" %f - %f * (1 - cos(t*3.14159/180)) + ( %f * cos(t*3.14159/180)) + ( -( %f - %f * ( 1 - cos(t*3.14159/180) ) ) + ( ( %f - %f * ( 1 - cos(t*3.14159/180)))^2 + %f * cos(t*3.14159/180) )^0.5 )" % (rcc,A9,t,rcc,A9,rcc,A9,A15))
print("y(t):")
print(" %f * sin(t*3.14159/180) + %f + ( %f * sin(t*3.14159/180) ) + ( -( %f - %f * ( 1 - cos(t*3.14159/180) ) ) + ( ( %f - %f * ( 1 - cos(t*3.14159/180) ) )^2 + %f * cos(t*3.14159/180) )^0.5 ) * tan(t*3.14159/180)" % (A10,A11,t,rcc,A10,rcc,A10,A15))
print("Parameters:")
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (theta-0.0001) )

Cooling Channel, outer:

Bell
x(t):
 t + ( 0.015075 * t / ( t^2 + 0.046650 )^0.5) + ( t / ( t^2 + 0.046650 )^0.5) * ( -t + (t^2 + 0.057086 * sin(2*3.14159 - arctan(-4.629941*t)))^(.5)) / (sin(2*3.14159-arctan(-4.629941*t)))
y(t):
 2.314970 *t^2 - 0.322342 + ( -0.015075 / ( 4.629941 *( t^2 + 0.046650 )^0.5)) + ( 1 / ( -4.629941 * (t^2+ 0.046650 )^0.5)) * ( -t + ( t^2 + 0.057086 * sin(2*3.14159-arctan(-4.629941*t)))^0.5) / ( sin(2*3.14159 - arctan(-4.629941*t)))
Parameters:
t1:
 0.414555
t2:
 0.621971

Rapid Expansion
x(t):
 0.544890 * (1- (.382/1.382) * cos(t*3.14159/180)) + ( 0.015075 * cos(t*3.14159/180)) + ( -( 0.544890 * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) ) + (( 0.544890 * (1 - (0.382)/(1.382) * cos(t*3.14159/180) ))^2 + 0.057086 * cos(t*3.14159/180))^0.5)
y(t):
 0.150614 * sin(t*3.14159/180) + (-0.015075 *sin(t*3.14159/180)) + ( -( 0.544890 * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) ) + ( (0.544890 * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) )^2 + 0.057086 * cos(t*

In [65]:
print("Nozzle surface, outer:\n" )

print("Bell")
print("x(t):")
print(" t + ( %f * t / ( t^2 + %f )^0.5) + ( t / ( t^2 + %f )^0.5) * ( -t + (t^2 + %f * sin(2*3.14159 - arctan(%f*t)))^(.5)) / (sin(2*3.14159-arctan(%f*t))) + ( %f * t / ( t^2 + %f )^0.5)" % (t,A12,A12,A15,-2*A1,-2*A1,tout,A12))
print("y(t):")
print(" %f *t^2 - %f + ( %f / ( %f *( t^2 + %f )^0.5)) + ( 1 / ( %f * (t^2+ %f )^0.5)) * ( -t + ( t^2 + %f * sin(2*3.14159-arctan(%f*t)))^0.5) / ( sin(2*3.14159 - arctan(%f*t))) + ( %f / ( %f *( t^2 + %f )^0.5))" % (A1,A2,-t,2*A1,A12,-2*A1,A12,A15,-2*A1,-2*A1,-tout,2*A1,A12))
print("Parameters:")
print("t1:")
print(" %f" % (X1+0.0001) )
print("t2:")
print(" %f\n" % (re-0.0001) )

print("Rapid Expansion")
print("x(t):")
print(" %f * (1- (.382/1.382) * cos(t*3.14159/180)) + ( %f * cos(t*3.14159/180)) + ( -( %f * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) ) + (( %f * (1 - (0.382)/(1.382) * cos(t*3.14159/180) ))^2 + %f * cos(t*3.14159/180))^0.5) + ( %f * cos(t*3.14159/180))" % (A3,t,A3,A3,A15,tout)     )
print("y(t):")
print(" %f * sin(t*3.14159/180) + (%f *sin(t*3.14159/180)) + ( -( %f * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) ) + ( (%f * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) )^2 + %f * cos(t*3.14159/180) )^0.5 ) * ( -tan(t*3.14159/180) ) + (%f *sin(t*3.14159/180))" % (A4,-t,A3,A3,A15,-tout)     )
print("Parameters:" )
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (thetaN-0.001) )

print("Rapid Contraction")
print("x(t):")
print(" %f * (1 - 0.6 * cos(t*3.14159/180)) + ( %f * cos(t*3.14159/180)) + ( -( %f * ( 1 - 0.6 * cos(t*3.14159/180) ) ) + ( ( %f * ( 1 - 0.6 * cos(t*3.14159/180) ) )^2 + ( %f ) * cos(t*3.14159/180) )^0.5 ) + ( %f * cos(t*3.14159/180))" % (A5,t,A5,A5,A15,tout)     )
print("y(t):")
print(" %f * sin(t*3.14159/180) + (%f *sin(t*3.14159/180)) + ( -( %f * ( 1 - 0.6 * cos(t*3.14159/180) ) ) + ( ( %f * ( 1 - 0.6 * cos(t*3.14159/180) ) )^2 + %f * cos(t*3.14159/180) )^0.5 ) * ( tan(t*3.14159/180) ) + ( %f *sin(t*3.14159/180))"             % (A6,t,A5,A5,A15,tout)     )
print("Parameters:" )
print("t1:")
print(" 0.001" )
print("t2:")
print(" %f\n" % (theta-0.001) )

A21 = math.cos(math.radians(theta))*tout
A22 = math.sin(math.radians(theta))*tout

print("Linear contraction")
print("x(t):")
print(" t + %f + ( -t + ( t^2 + %f )^0.5 ) + %f" % (A13,A16,A21))
print("y(t):")
print(" %f *t + %f + %f + ( -t + ( t^2 + %f )^0.5 ) * ( %f ) + ( %f ) + ( %f )" % (A7,A17,A18,A19,A20,A14,A22))
print("Parameters:" )
print("t1:")
print(" %f" % (p1+0.0001) )
print("t2:")
print(" %f\n" % (p2-0.0001) )

print("Inlet")
print("x(t):")
print(" %f - %f * (1 - cos(t*3.14159/180)) + ( %f * cos(t*3.14159/180)) + ( -( %f - %f * ( 1 - cos(t*3.14159/180) ) ) + ( ( %f - %f * ( 1 - cos(t*3.14159/180)))^2 + %f * cos(t*3.14159/180) )^0.5 ) + ( %f * cos(t*3.14159/180))" % (rcc,A9,t,rcc,A9,rcc,A9,A15,tout))
print("y(t):")
print(" %f * sin(t*3.14159/180) + %f + ( %f * sin(t*3.14159/180) ) + ( -( %f - %f * ( 1 - cos(t*3.14159/180) ) ) + ( ( %f - %f * ( 1 - cos(t*3.14159/180) ) )^2 + %f * cos(t*3.14159/180) )^0.5 ) * tan(t*3.14159/180) + (%f *sin(t*3.14159/180))" % (A10,A11,t,rcc,A10,rcc,A10,A15,tout))
print("Parameters:")
print("t1:")
print(" 0.001"  )
print("t2:")
print(" %f\n" % (theta-0.0001) )

Nozzle surface, outer:

Bell
x(t):
 t + ( 0.015075 * t / ( t^2 + 0.046650 )^0.5) + ( t / ( t^2 + 0.046650 )^0.5) * ( -t + (t^2 + 0.057086 * sin(2*3.14159 - arctan(-4.629941*t)))^(.5)) / (sin(2*3.14159-arctan(-4.629941*t))) + ( 0.030151 * t / ( t^2 + 0.046650 )^0.5)
y(t):
 2.314970 *t^2 - 0.322342 + ( -0.015075 / ( 4.629941 *( t^2 + 0.046650 )^0.5)) + ( 1 / ( -4.629941 * (t^2+ 0.046650 )^0.5)) * ( -t + ( t^2 + 0.057086 * sin(2*3.14159-arctan(-4.629941*t)))^0.5) / ( sin(2*3.14159 - arctan(-4.629941*t))) + ( -0.030151 / ( 4.629941 *( t^2 + 0.046650 )^0.5))
Parameters:
t1:
 0.414555
t2:
 0.621971

Rapid Expansion
x(t):
 0.544890 * (1- (.382/1.382) * cos(t*3.14159/180)) + ( 0.015075 * cos(t*3.14159/180)) + ( -( 0.544890 * ( 1 - (0.382)/(1.382) * cos(t*3.14159/180) ) ) + (( 0.544890 * (1 - (0.382)/(1.382) * cos(t*3.14159/180) ))^2 + 0.057086 * cos(t*3.14159/180))^0.5) + ( 0.030151 * cos(t*3.14159/180))
y(t):
 0.150614 * sin(t*3.14159/180) + (-0.015075 *sin(t*3.14159/180)) + ( -( 0.544890 * (